In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings(action='ignore')
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances

In [3]:
upper1=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의1.xlsx',thousands = ',')
upper2=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의2.xlsx',thousands = ',')
lower1=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의1.xlsx',thousands = ',')
lower2=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의2.xlsx',thousands = ',')

def remove(a):
    a.price=a.price.str.replace('\n','')
    a.tag=a.tag.str.replace('\n','')
    a=a.replace('Null',np.nan)
    return a

upper1=remove(upper1)
upper2=remove(upper2)
lower1=remove(lower1)
lower2=remove(lower2)

# 모든 컨텐츠데이터 불러온 후 하나의 데이터프레임으로 만들기
df=pd.concat([upper1,upper2,lower1,lower2]).reset_index()

# 태그 null값인거 제거 
DF=df[df['tag'].notnull()]
DF.reset_index(inplace=True)
DF.drop(['level_0','index'],axis=1,inplace=True)

# 태그를 # 기준으로 나누기
DF.tag=DF.tag.str.split('#')

#앞의 , 없애주기
for i in range(DF.shape[0]):
    del DF.tag[i][0]

# item_name에 붙어있는 색상 일부분제거 - 추후 개선 필요
DF['item_name']=DF.item_name.str.replace(r'\[[^)]*\]', '')
DF['item_name']=DF.item_name.str.replace(r'\([^)]*\)', '')

# 중복되는 item_name 제거 
DF=DF.drop_duplicates(['item_name'])
DF.reset_index(inplace=True)
DF.drop('index',axis=1,inplace=True)

# 수치형 변수 데이터 타입 변경
DF=DF.astype({'score_average':'float'})
DF=DF.astype({'review_count':'float'})
DF.describe()

,score_average,review_count,like_count
count,1945.000000,1646.000000,2625.000000
mean,4.714859,4249.758809,3219.206476
std,0.250247,7804.381531,6434.393090
min,2.000000,10.000000,0.000000
25%,4.700000,290.000000,29.000000
50%,4.700000,1436.500000,758.000000
75%,4.800000,4659.750000,3710.000000
max,5.000000,104967.000000,72628.000000


In [4]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_name      3125 non-null   object 
 1   brand          3125 non-null   object 
 2   score_average  1945 non-null   float64
 3   review_count   1646 non-null   float64
 4   like_count     2625 non-null   float64
 5   tag            3125 non-null   object 
 6   price          3125 non-null   object 
dtypes: float64(3), object(4)
memory usage: 171.0+ KB


## 가중 평균


- r : 개별 옷에 평점을 준 횟수 
- m : 평점을 부여하기 위한 최소 리뷰 개수 
- R : 개별 영화에 대한 평균평점
- M : 전체 영화에 대한 평균평점

## TFIDF

## CountVectorizer

## Word2vec

## 가중치

In [11]:
# 가중평균함수 
# 리뷰개수와 평균평점으로 가중치 적용
# 리뷰 개수가 적고 평균평점이 높을 경우 평점이 높이 반영되는 경우를 막기 위해
def weighted_vote_average(record):
    r=record['review_count']
    R=record['score_average'] 
    return ( (r/(r+m)*R) ) + ( (m/(m+r)*M)  )

M=np.round(DF['score_average'].mean(),4) # 전체 평점 평균
m=DF['review_count'].quantile(0.5)       # 리뷰 조회수 50%


DF['weighted_vote']=DF.apply(weighted_vote_average,axis=1) # 함수적용해서 열방향으로 붙혀주기

DF[['item_name','score_average','review_count','weighted_vote']].sort_values( by='weighted_vote',ascending=False)

# TFIDF
DF.tag_tfidf=DF.tag.apply(lambda x : (' ').join(x))
DF['tag_tfidf']=DF.tag_tfidf

tfidf=TfidfVectorizer(min_df=1,ngram_range=(1,3))
tag_tf=tfidf.fit_transform(DF['tag_tfidf'])

# 1. 코사인유사도
tf_sim=cosine_similarity(tag_tf,tag_tf)
# 2. 유클리디언 유사도
tf_euc=euclidean_distances(tag_tf,tag_tf)
# 3. 맨하탄 유사도 
tf_man=manhattan_distances(tag_tf,tag_tf)


# CountVectorizer
count_vect=CountVectorizer(min_df=0,ngram_range=(1,3))
tag_count=count_vect.fit_transform(DF['tag_tfidf'])

# 1. 코사인유사도
ct_sim=cosine_similarity(tag_count,tag_count)
# 2. 유클리디언 유사도
ct_euc=euclidean_distances(tag_count,tag_count)
# 3. 맨하탄 유사도 
ct_man=manhattan_distances(tag_count,tag_count)



# Word2vec 
word=[]
for words in DF.tag:
    word.append(words)

word2vec=Word2Vec(sentences=word,window=5,sg=1,min_count=1,vector_size=100,epochs=100)


# 단어 벡터 평균구하기
def vectors(embedding):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(word2vec.wv[word]) 
            count+=1
        except : continue
    
    
    word_embedding2=np.array(word_embedding)
    
    
    v=word_embedding2.sum(axis=0)  
    
    
    if type(v)!=np.ndarray:
        return np.zeors(100)
    
    return(v/count)


# 하나의 백터값 확인
np.array(word2vec.wv['기본티']) 
# 모든 
wordvec=[vectors(x) for x in word]

# 1. 코사인유사도
w2v_sim=cosine_similarity(wordvec,wordvec)
# 2. 유클리디언 유사도
w2v_euc=euclidean_distances(wordvec,wordvec)
# 3. 맨하탄 유사도 
w2v_man=manhattan_distances(wordvec,wordvec)


# 가중치 부여 vec
new_vec=(w2v_sim+tf_sim)/ 2


#최종 알고리즘
def sim_clothes(clothe,sim,top_n):
    
    # 찾고자 하는 옷선택
    clothes=DF[DF.item_name==clothe]
    # 찾고자 하는 옷의 인덱스 값만 추출하여 새로운 변수에 저장
    clothes_index=clothes.index.values
    
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]
    
    # 내가 추출하려는 옷의 인덱스의 행번째추출, 이중리스트이므로 [0][0]
    # -> 아이템이름을 추출 -> 내가 원하는 옷의 이름과 같지 않다면
    if DF.iloc[sim_sorted_ind[clothes.index][0][0]].item_name != clothe:
        
        # 내가 추출하려는 옷의 인덱스 행번째의 array를 리스트로 변경, 변수명 l로 저장
        l=sim_sorted_ind[clothes.index][0].tolist()
        # 내가 추출하려는 옷의 인덱스 값 자체를 리스트로 변경한 후 값으로 추출
        v=clothes.index.values.tolist()[0]
        
        # 리스트 l 에서 내가 추출하려고 한 옷의 값을 제거
        # -> 내가 추출하고자 하는 옷의 인덱스 번호가 제거된 리스트 생성
        l.remove(clothes.index.values.tolist()[0])
        
        # 리스트l의 array형태로 맞춰주기
        sim_sorted_ind2=np.array(l)
        
    
        # 자기 자신을 제외하고 유사도가 높은 인덱스를 top_n개수 만큼 추출
        # 내가 추출하고자 하는 인덱스의 값이 제거된 상태이므로 0값부터 원하는 개수만큼 출력
        similar_indexes=sim_sorted_ind2[0:(top_n)]
        a=  DF.iloc[similar_indexes].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    else :
        # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
        similar_indexes=sim_sorted_ind[clothes_index,1:(top_n+1)]
        a= DF.iloc[similar_indexes[0]].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    return a

In [12]:
sim_clothes('PFP 체크 팬츠 그린',new_vec,5)

,item_name,brand,score_average,review_count,like_count,tag,price,weighted_vote
486,쉐도우 탱크 - 블랙 / FU6005,ADIDAS,4.8,309.0,515.0,"[로고티, 로고프린트, 쉐도우트레포일, 탱크티, 트레포일티]","39,000원",4.729965
3124,타이다이 부츠컷 링클 팬츠 블랙,PRENDA FROM PLANT,NaN,NaN,24.0,[21SUMMER컬렉션],"65,000원",NaN
3120,타이다이 부츠컷 링클 팬츠 블루,PRENDA FROM PLANT,NaN,NaN,4.0,[21SUMMER컬렉션],"65,000원",NaN
1409,로즈 골지 스트라이프 니트,CLANGCLANG,NaN,NaN,NaN,"[골프, 여성골프웨어, 골프웨어, 골프니트, 여자니트, 터틀넥니트, 여성니트]","99,000원",NaN
1358,브라운 헨리넥 반팔 니트_릴렉스핏,8SECONDS,NaN,NaN,NaN,[421551LYZD],"35,900원",NaN


In [5]:
DF

,item_name,brand,score_average,review_count,like_count,tag,price,weighted_vote,tag_tfidf
0,릴렉스 핏 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.7,104967.0,34769.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션,...","11,900원",4.700201,기본티 라운드넥 루즈핏 면티셔츠 무지티 반팔티 MU-T 홈웨어컬렉션 유아인착장상품 베이식
1,쿨 코튼 2-PACK 티셔츠,COVERNAT,4.8,4315.0,42677.0,"[라운드티, 로고티셔츠, 반팔티셔츠, 티셔츠, 반팔, 반팔티, 반팔셔츠]","43,000원",4.778745,라운드티 로고티셔츠 반팔티셔츠 티셔츠 반팔 반팔티 반팔셔츠
2,NYC LOCATION T-SHIRT,GROOVERHYME,4.7,17893.0,47049.0,"[로고, 맨투맨, 반팔티, 오버사이즈, 오버핏, 티셔츠, 패키지]","58,000원",4.701107,로고 맨투맨 반팔티 오버사이즈 오버핏 티셔츠 패키지
4,빅 트위치 로고 티셔츠 화이트,LEE,4.7,9374.0,50486.0,"[로고, 반팔티, 빅로고, LEE반팔티, 트위치, 남녀공용]","35,000원",4.701980,로고 반팔티 빅로고 LEE반팔티 트위치 남녀공용
5,레이어드 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.6,14441.0,31470.0,"[기본티셔츠, 라운드넥, 무지티셔츠, MU-T, 홈웨어컬렉션]","13,900원",4.610395,기본티셔츠 라운드넥 무지티셔츠 MU-T 홈웨어컬렉션
...,...,...,...,...,...,...,...,...,...
3928,타이다이 부츠컷 링클 팬츠 블루,PRENDA FROM PLANT,NaN,NaN,4.0,[21SUMMER컬렉션],"65,000원",NaN,21SUMMER컬렉션
3929,플리츠 스트레이트 팬츠_차콜,ENOU,4.9,40.0,53.0,"[팬츠, 바지, 주름, 플리츠, 플리츠팬츠]","79,000원",4.719915,팬츠 바지 주름 플리츠 플리츠팬츠
3930,PFP 체크 팬츠 그린,PRENDA FROM PLANT,NaN,NaN,0.0,[21SUMMER컬렉션],"65,000원",NaN,21SUMMER컬렉션
3932,마리아 골프 팬츠 2101335036110015,J.LINDEBERG,NaN,NaN,1.0,"[WOMEN, 마리아, 골프, 팬츠]","339,000원",NaN,WOMEN 마리아 골프 팬츠


1. 지금 앙상블을 할라고 했는데 어떻게 확률값을 구해야할지 몰라서 그냥 모델 두개의 코사인 유사도 값을 더해서 추천 모델을 돌림 ( 이게 과연 앙상블인가?)


- 쨋든 돌렸는데 태그가 같은 상품들은 상품이름이 달라도 같은것으로 인식해서 추천 잘 못함


- 라이크 카운트로나마 해결책을 찾고자 함


- ex) 유사도 벡터에다가 로그를 씌운 라이크 카운트를 곱해주거나 더해주거나..
    근데 결과가 거지같음

    ** 유사도벡터는 건들면 안된다
    
2. 우리의 문제점 : 태그가 같은 값들은 상품을 구분 못함 어떻게 함?